This code is a pipeline for generating GPU prices based on eBay listings for various specified GPU models. This code is meant to be ran locally for collection and storage and does not require a large amount of computation power in order to do so. It is also meant to be ran daily for building the database.

This is the first portion of our pipeline: webscraping eBay to collect listing data.

In [ ]:
import time
import re
import os
import datetime
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver as uc

# Set working directory (optional, you might remove this from the module)
os.chdir("C:/code/python/ECEN360/final_project")

def parse_price(price_text):
    price_text = price_text.replace(",", "")
    match = re.search(r"\$([\d\.]+)", price_text)
    return float(match.group(1)) if match else 0.0

def scrape_ebay_selenium(search_query, max_pages=1, only_used=False, filter=None, return_df=True):
    filter = filter or []

    options = uc.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = uc.Chrome(
        driver_executable_path=ChromeDriverManager().install(),
        options=options
    )

    driver.get("https://www.ebay.com/")
    time.sleep(2)

    search_box = driver.find_element(By.NAME, "_nkw")
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

    results = []

    for page in range(1, max_pages + 1):
        print(f"Scraping Page {page}: {driver.current_url}")

        items = driver.find_elements(By.CSS_SELECTOR, "li.s-item")
        item_data = []

        for item in items:
            try:
                title = item.find_element(By.CSS_SELECTOR, ".s-item__title").text
                price = item.find_element(By.CSS_SELECTOR, ".s-item__price").text
                link = item.find_element(By.CSS_SELECTOR, ".s-item__link").get_attribute("href")
                condition = item.find_element(By.CSS_SELECTOR, ".SECONDARY_INFO").text

                try:
                    shipping = item.find_element(By.CSS_SELECTOR, ".s-item__shipping").text
                except:
                    shipping = "$0.00"

                title_lower = title.lower()
                if any(bad in title_lower for bad in filter):
                    continue
                if "for parts or not working" in condition.lower():
                    continue
                if only_used and "used" not in condition.lower():
                    continue

                if item.find_elements(By.CSS_SELECTOR, ".s-item__bidCount"):
                    continue

                item_price = parse_price(price)
                shipping_price = parse_price(shipping)
                total_price = item_price + shipping_price

                item_data.append({
                    "Title": title,
                    "Link": link,
                    "Total Price": round(total_price, 2)
                })

            except Exception:
                continue

        for entry in item_data:
            try:
                driver.get(entry["Link"])
                time.sleep(2)

                try:
                    description_elem = driver.find_element(By.ID, "desc_ifr")
                    driver.switch_to.frame(description_elem)
                    item_description = driver.find_element(By.XPATH, "//body").text
                    driver.switch_to.default_content()
                except:
                    item_description = ""

                entry["Item Description"] = item_description
                results.append(entry)

            except Exception:
                continue

        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.pagination__next")
            next_button.click()
            time.sleep(3)
        except:
            print("No more pages available.")
            break

    driver.quit()

    df = pd.DataFrame(results, columns=["Title", "Item Description", "Total Price", "Link"])
    print(len(df), "items scraped.")
    return df if return_df else df.to_csv(f"{search_query}_{datetime.datetime.today().strftime('%Y-%m-%d')}.csv", index=False)


#scrape_ebay_selenium("RTX 5060", max_pages=3, only_used=False, filter=["5060ti", "5060 ti", "5070", "5080", "5090"], return_df=False)

This section inputs our title and listing description to be filtered by a small LM for more advanced filtering.

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import os

# Optional: Set working directory if needed
os.chdir("C:/code/python/ECEN360/final_project")

# Load model once globally
model = SentenceTransformer('all-MiniLM-L6-v2')

def classify_and_filter(
    df,
    positive_prompt,
    negative_prompt,
    threshold=0.5,
    margin=0.1,
    save_filtered=False,
    save_path=None
):
    # Combine title and description for similarity comparison
    texts = (df['Title'].fillna('') + ". " + df['Item Description'].fillna('')).tolist()

    # Encode prompts and texts
    pos_embed = model.encode(positive_prompt, convert_to_tensor=True)
    neg_embed = model.encode(negative_prompt, convert_to_tensor=True)
    text_embeddings = model.encode(texts, convert_to_tensor=True)

    # Calculate similarities
    pos_sims = util.cos_sim(pos_embed, text_embeddings)[0].cpu().numpy()
    neg_sims = util.cos_sim(neg_embed, text_embeddings)[0].cpu().numpy()

    # Add similarity scores to DataFrame
    df = df.copy()
    df['Positive Similarity'] = pos_sims
    df['Negative Similarity'] = neg_sims
    df['Delta'] = df['Positive Similarity'] - df['Negative Similarity']

    # Apply filters
    filtered_df = df[(df['Positive Similarity'] >= threshold) & (df['Delta'] >= margin)]

    if save_filtered and save_path:
        filtered_df.to_csv(save_path, index=False)
        print(f"Filtered DataFrame saved to: {save_path}")

    print (len(filtered_df), "items after filtering")
    return filtered_df

#classify_and_filter(df = pd.read_csv("RX 7900 XT_2025-04-23.csv"), positive_prompt="fully functional RX 7900 XT graphics card GPU", negative_prompt="7900 XTX, 7900XTX", threshold=0.6, margin=0.1, save_filtered=True, save_path="filtered_RX_7900_XT.csv")


This section takes our filtered listings and processes them to find an average price and append it the database.

In [ ]:
import pandas as pd
import os

# Optional: Set working directory
os.chdir("C:/code/python/ECEN360/final_project")

def filter_and_mean_price(data):
    # Determine if input is a file path or DataFrame
    if isinstance(data, str):
        df = pd.read_csv(data)
    elif isinstance(data, pd.DataFrame):
        df = data.copy()
    else:
        raise ValueError("Input must be a file path or a pandas DataFrame.")

    # Clean the 'Total Price' column: remove non-numeric characters
    df['Total Price'] = df['Total Price'].replace({r'[^\d.]': ''}, regex=True)
    df['Total Price'] = pd.to_numeric(df['Total Price'], errors='coerce')

    # Report NaN count
    print(f"Number of NaN values in 'Total Price' column: {df['Total Price'].isna().sum()}")

    # Drop rows with NaN in 'Total Price'
    df_cleaned = df[df['Total Price'].notna()]

    # Safety check for minimum data
    if len(df_cleaned) < 5:
        return "Not enough data to perform filtering and mean calculation."

    # Sort and trim extremes
    df_sorted = df_cleaned.sort_values(by='Total Price')
    n = len(df_sorted)
    lower_idx = int(n * 0.2)
    upper_idx = int(n * 0.8)
    df_filtered = df_sorted.iloc[lower_idx:upper_idx]

    if len(df_filtered) == 0:
        return "After filtering, no data remains to calculate the mean."

    # Calculate mean
    return df_filtered['Total Price'].mean()

#filter_and_mean_price("filtered_RX_7900_XT.csv")


This final section combines all of the above programs into one program to run locally. We can also add or remove specified GPU targets along with the necessary filters and parameters in this program. This will automatically collect the data and store it into the database once ran.

In [ ]:
import os
import pandas as pd
from datetime import datetime


# Set working directory
os.chdir("C:/code/python/ECEN360/final_project")

# Confirm it worked
print("Current working directory:", os.getcwd())

from scrape import scrape_ebay_selenium
from one_shot import classify_and_filter
from filter_and_find_mean import filter_and_mean_price



class GPUQuery:
    def __init__(self, name, filter_terms=None, positive_prompt="", negative_prompt="", threshold=0.4, margin=0.1):
        self.name = name
        self.filter_terms = filter_terms or []
        self.positive_prompt = positive_prompt
        self.negative_prompt = negative_prompt
        self.threshold = threshold
        self.margin = margin
        self.data = None  # This will store the DataFrame after scraping

    def run_pipeline(self, max_pages=3, only_used=False):
        print(f"\nRunning pipeline for {self.name}...")

        # 1. Scrape eBay listings
        self.data = scrape_ebay_selenium(
            self.name,
            max_pages=max_pages,
            only_used=only_used,
            filter=self.filter_terms,
            return_df=True
        )

        # 2. Apply one-shot classification
        self.data = classify_and_filter(
            self.data,
            negative_prompt=self.negative_prompt,
            positive_prompt=self.positive_prompt,
            threshold=self.threshold,
            margin=self.margin
        )

        # 3. Filter by price and calculate mean
        mean_price = filter_and_mean_price(self.data)
        print(mean_price)
        try:
            return round(mean_price, 2)
        except:
            return 0

def log_gpu_price(gpu_name, mean_price, log_filename="gpu_price_log.csv"):
    # Today's date in YYYY-MM-DD format
    today = datetime.today().strftime('%Y-%m-%d')

    # Prepare entry as a DataFrame row
    new_entry = pd.DataFrame([{
        "Date": today,
        "GPU": gpu_name,
        "Mean Price (eBay)": round(mean_price, 2) if isinstance(mean_price, float) else None
    }])

    # If file exists, append; otherwise, create it
    if os.path.exists(log_filename):
        log_df = pd.read_csv(log_filename)
        updated_df = pd.concat([log_df, new_entry], ignore_index=True)
    else:
        updated_df = new_entry

    # Save updated log
    updated_df.to_csv(log_filename, index=False)
    print(f"Logged: {gpu_name} at ${new_entry['Mean Price (eBay)'].iloc[0]} on {today}")

gpus = [
    GPUQuery(
        name="RTX 4060",
        filter_terms=["4060 ti", "4060ti", "4070", "4070super"],
        positive_prompt="fully functional RTX regular 4060 8GB graphics card GPU",
        negative_prompt="Ti, broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RTX 4060 Ti",
        filter_terms=["4070 ti", "4070ti", "4070 super", "4070super"],
        positive_prompt="fully functional RTX 4060 Ti 8GB graphics card GPU",
        negative_prompt="regular 4060, non-Ti"
    ),
    GPUQuery(
        name="RTX 4070",
        filter_terms=["4070 ti", "4070ti", "4070 super", "4070super"],
        positive_prompt="fully functional RTX 4070 graphics card GPU",
        negative_prompt="4070 SUPER, 4070 Ti"
    ),
    GPUQuery(
        name="RTX 4070 Ti",
        filter_terms=["4070 super", "4070super", "4070 ti super", "4070ti super"],
        positive_prompt="fully functional RTX 4070 Ti graphics card GPU",
        negative_prompt="super, non-ti"
    ),
    GPUQuery(
        name="RTX 4070 Super",
        filter_terms=["4070 ti", "4070ti", "4070Ti super", "4070 ti super"],
        positive_prompt="fully functional RTX 4070 Super graphics card GPU",
        negative_prompt="Ti, super"
    ),
    GPUQuery(
        name="RTX 4070 Ti Super",
        filter_terms=["non-super", "non ti", "12GB" "12 GB"],
        positive_prompt="fully functional RTX 4070 Ti Super graphics card GPU",
        negative_prompt="non-super, non-ti, relugar, broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RTX 4080",
        filter_terms=["4080 super", "4080super", "4080 ti", "4080ti"],
        positive_prompt="fully functional RTX 4080 graphics card GPU",
        negative_prompt="SUPER"
    ),
    GPUQuery(
        name="RTX 4080 Super",
        filter_terms=["4080 ti, non-super"],
        positive_prompt="fully functional RTX 4080 Super graphics card GPU",
        negative_prompt="Regular, Ti, non-super"
    ),
    GPUQuery(
        name="RTX 4090",
        filter_terms=["4080, broken, non-functional, for parts, not working"],
        positive_prompt="fully functional RTX 4090 graphics card GPU",
        negative_prompt="broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RX 7900 XTX",
        filter_terms=["7900 XT", "7900XT,", "7900GRE", "7900 GRE"],
        positive_prompt="fully functional RX 7900 XTX graphics card GPU",
        negative_prompt="7900XTX, 7900 XT, 7900GRE, 7900 GRE",
    ),
    GPUQuery(
        name="RX 7900 GRE",
        filter_terms=["7900 XT", "7900XT", "7900XTX", "7900 XTX"],
        positive_prompt="fully functional RX 7900 GRE graphics card GPU",
        negative_prompt="7900XTX, 7900 XT, 7900XT",
    ),
    GPUQuery(
        name="RX 7900 XT",
        filter_terms=["7900XTX", "7900 XTX", "7900GRE", "7900 GRE"],
        positive_prompt="fully functional RX 7900 XT graphics card GPU",
        negative_prompt="7900 XTX, 7900XTX",
    ),
    GPUQuery(
        name="RX 7800 XT",
        filter_terms=["7900XTX", "7900 XTX", "7900GRE", "7900 GRE"],
        positive_prompt="fully functional RX 7800 XT graphics card GPU",
        negative_prompt="7900 XTX, 7900XTX",
    ),
    GPUQuery(
        name="RX 7700 XT",
        filter_terms=["7600", "7800", "7900"],
        positive_prompt="fully functional RX 7700 XT graphics card GPU",
        negative_prompt="7600, 7600 XT, 7600XT"
    ),
    GPUQuery(
        name="RX 7600 XT",
        filter_terms=["7700", "7800", "7900"],
        positive_prompt="fully functional RX 7600 XT graphics card GPU",
        negative_prompt="regular 7600, non-XT"
    ),GPUQuery(
        name="RX 7600",
        filter_terms=["7600XT", "7600 XT", "7700"],
        positive_prompt="fully functional regular non-XT RX 7600 graphics card GPU",
        negative_prompt="7600 XT, 7600XT, 7700"
    ),
    GPUQuery(
        name="RTX 5060",
        filter_terms=["5060ti", "5060 ti", "5070", "5080", "5090"],
        positive_prompt="fully functional regular non-Ti RTX 5060 graphics card GPU",
        negative_prompt="5060 Ti, 5060Ti"
    ),
    GPUQuery(
        name="RTX 5060 Ti",
        filter_terms=["5070", "5080", "5090"],
        positive_prompt="fully functional RTX 5060 Ti graphics card GPU",
        negative_prompt="regular 5060, non-Ti, broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RTX 5070",
        filter_terms=["5060", "5060 ti", "5070Ti", "5070 ti", "5090", "5080"],
        positive_prompt="fully functional regular, non-Ti RTX 5070 graphics card GPU",
        negative_prompt="5070 Ti, broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RTX 5070 Ti",
        filter_terms=["5060", "5060 ti", "5090", "5080"],
        positive_prompt="fully functional RTX 5070 Ti graphics card GPU",
        negative_prompt="regular 5070, non-Ti, broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RTX 5080",
        filter_terms=["5060", "5080 ti", "5080ti", "5080 super", "5080super", "5090", "5070"],
        positive_prompt="fully functional RTX 5080 graphics card GPU",
        negative_prompt="broken, non-functional, for parts, not working"
    ),
    GPUQuery(
        name="RTX 5090",
        filter_terms=["5060", "5080", "5070"],
        positive_prompt="fully functional RTX 5090 graphics card GPU",
        negative_prompt="broken, non-functional, for parts, not working"
    ),
]

for gpu in gpus:
    mean_price = gpu.run_pipeline(max_pages=3, only_used=False)
    log_gpu_price(gpu.name, mean_price)
